## Basic scraper for bill list

From new state legislative search (earliest session 120)

In [1]:
import pandas as pd
import parse_bills as pb
import requests
import sqlite3 as sql

In [2]:
cur_leg = requests.get('https://legislature.maine.gov/backend/breeze/data/getCurrentLegislature').json()[0]
cur_leg

131

## First legislature fixed at 120

In [3]:
start_leg = 131
leg_list = list(range(start_leg, cur_leg + 1))
leg_list

[131]

In [4]:
from importlib import reload
reload(pb)
urls = pb.generate_query_list(session_list=leg_list)

https://legislature.maine.gov/mrs-search/api/billtext?term=&title=&legislature=131&requestItemType=false&lmSponsorPrimary=false&reqAmendExists=false&reqAmendAdoptH=false&reqAmendAdoptS=false&reqChapterExists=false&reqFNRequired=false&reqEmergency=false&reqGovernor=false&reqBond=false&reqMandate=false&reqPublicLand=false&showExtraParameters=true&mustHave=&mustNotHave=&offset=0&pageSize=20&sortByScore=false&showBillText=false&sortAscending=false&excludeOrders=false
Legislature 131
Generated 258 URLs | page size: 20 | results: 5160


Generated 258 URLs


In [17]:
r = await pb.run_in_batches(10, urls)

Fetching Data:  21%|██        | 107/516 [00:06<00:23, 17.50query/s]


CancelledError: 

In [ ]:
src = [y[0]['_source'] for y in [x['hits']['hits'] for x in r]]
df = pd.json_normalize(src)
conn = sql.connect('data/maine_legislation.db')
df.to_sql('maine_bill_text', conn, if_exists='replace', index=False)
conn.close()

## Format testimony search from bill

## Query Paper Numbers and session for testimony results


In [19]:
conn = sql.connect('data/maine_legislation.db')
query = '''
SELECT DISTINCT paperNumber, legislature
FROM maine_bill_text mbt 
WHERE paperNumber IS NOT NULL 
ORDER BY legislature, paperNumber
'''
doc_leg_list = conn.execute(query).fetchall()
conn.close()

In [20]:
conn = sql.connect('data/maine_legislation.db')
pd.read_sql(query, conn).head()

,paperNumber,legislature
0,HP0006,120
1,HP0010,120
2,HP0015,120
3,HP0021,120
4,HP0023,120


In [22]:
from urllib.parse import urlencode

base_url = 'https://legislature.maine.gov/backend/breeze/data/CommitteeTestimony'

query_params = [{
        '$filter': (
            f"(((Request/PaperNumber eq '{paper_number}') and "
            f"(Request/Legislature eq {legislature})) and "
            "(Inactive ne true)) and "
            # f"(PresentedDate lt datetime'{presented_date}') and "  # Uncomment if needed
            "(not (startswith(LastName, '@') eq true))"
        ),
        '$orderby': 'LastName,FirstName,Organization',
        '$expand': 'Request',
        '$select': 'Id,SourceDocument,RequestId,FileType,FileSize,NamePrefix,FirstName,LastName,NameSuffix,Organization,PresentedDate,PolicyArea,Topic,Created,CreatedBy,LastEdited,LastEditedBy,Private,Inactive,TestimonySubmissionId,HearingDate,LDNumber,Request,CommitteeTestimonyDocumentContents'
    }
    for paper_number, legislature in doc_leg_list
]

urls = [f'{base_url}?{urlencode(query_param)}' for query_param in query_params]

# Encode query parameters into a URL
testimonies = await pb.run_in_batches(15, urls)

Fetching Data:  55%|█████▍    | 3193/5832 [03:41<10:23,  4.23query/s]ERROR:root:Error fetching https://legislature.maine.gov/backend/breeze/data/CommitteeTestimony?%24filter=%28%28%28Request%2FPaperNumber+eq+%27SP0601%27%29+and+%28Request%2FLegislature+eq+125%29%29+and+%28Inactive+ne+true%29%29+and+%28not+%28startswith%28LastName%2C+%27%40%27%29+eq+true%29%29&%24orderby=LastName%2CFirstName%2COrganization&%24expand=Request&%24select=Id%2CSourceDocument%2CRequestId%2CFileType%2CFileSize%2CNamePrefix%2CFirstName%2CLastName%2CNameSuffix%2COrganization%2CPresentedDate%2CPolicyArea%2CTopic%2CCreated%2CCreatedBy%2CLastEdited%2CLastEditedBy%2CPrivate%2CInactive%2CTestimonySubmissionId%2CHearingDate%2CLDNumber%2CRequest%2CCommitteeTestimonyDocumentContents: Unterminated string starting at: line 1 column 1455109 (char 1455108)
ERROR:root:Error fetching https://legislature.maine.gov/backend/breeze/data/CommitteeTestimony?%24filter=%28%28%28Request%2FPaperNumber+eq+%27SP0601%27%29+and+%28Request%

CancelledError: 

In [ ]:
testimony_df = pd.json_normalize(testimonies)

In [ ]:
conn = sql.connect('data/maine_legislation.db')
testimony_df.to_sql('maine_bill_testimony', conn, if_exists='replace', index=False)
conn.close()

## All Testimony

In [23]:
all_url = 'https://legislature.maine.gov/backend/breeze/data/CommitteeTestimony'
all_results = requests.get(all_url)

In [28]:
import json
all_df = pd.json_normalize(json.loads(all_results.content))
len(all_df)

137227

In [29]:
# src = [y[0]['_source'] for y in [x['hits']['hits'] for x in r]]
conn = sql.connect('data/maine_legislation.db')
all_df.to_sql('maine_bill_testimony', conn, if_exists='replace', index=False)
conn.close()

In [21]:
all_df.columns

Index(['$id', '$type', 'Id', 'SourceDocument', 'RequestId', 'FileType',
       'FileSize', 'NamePrefix', 'FirstName', 'LastName', 'NameSuffix',
       'Organization', 'PresentedDate', 'PolicyArea', 'Topic', 'Created',
       'CreatedBy', 'LastEdited', 'LastEditedBy', 'Private', 'Inactive',
       'TestimonySubmissionId', 'HearingDate', 'LDNumber', 'Request',
       'CommitteeTestimonyDocumentContents'],
      dtype='object')

## GET testimony doc from ID

In [ ]:
doc_id = 10031266

doc_test = f'https://legislature.maine.gov/backend/app/services/getDocument.aspx?doctype=test&documentId={doc_id}'

requests.get()

In [36]:
list_url = f'{base_url}?{urlencode(query_params[4000])}'
list_url

In [37]:
requests.get(list_url).content

b'[{"$id":"1","$type":"_IB_1lt3YFZUpGY15A9kn7_pEYC1T5Fg[[System.Int32, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.DateTime, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.Boolean, mscorlib]], _IB_1lt3YFZUpGY15A9kn7_pEYC1T5Fg_IdeaBlade","Id":50406,"FileType":"pdf","NamePrefix":null,"FirstName":"Patrick","LastName":"Gagnon","NameSuffix":null,"Organization":"Maine State Law Enforcement Assn.","PresentedDate":"2017-03-27T00:00:00.000","FileSize":"379 KB","Topic":null,"Private":false},{"$id":"2","$type":"_IB_1lt3YFZUpGY15A9kn7_pEYC1T5Fg[[System.Int32, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.DateTime, mscorlib],[System.String, mscorlib],[System.String, mscorlib],[System.Boolean, mscorlib]], _IB_1lt3YFZUpGY15A

## Retrieve actual testimony document

In [ ]:
https://legislature.maine.gov/backend/app/services/getDocument.aspx?doctype=test&documentId=166895

## Bill actions summary

In [ ]:
from urllib.parse import urlencode

base_url = 'https://legislature.maine.gov/backend/breeze/data/Request'

legislature = 131
paper_number = 'SP0008'

query_params = {
    '$filter': f"(Legislature eq {legislature}) and (PaperNumber eq '{paper_number}')",
    '$expand': (
        'ActionFinalDisp,ActionCouncil,ActionGovernor,ActionRequest,ActionVeto,'
        'ActionCommittee,Committee1stRef,Committee2ndRef,CommitteeFrom,RequestItem,'
        'RequestTitleSections,RequestAction'
    )
}

# Encode query parameters into a URL
full_url = f"{base_url}?{urlencode(query_params)}"

full_url

In [ ]:
actions = requests.get(full_url)
actions.json()

In [ ]:
pd.DataFrame([x['ActionFinalDisp'] for x in actions.json()])

## Asyncio

In [ ]:
import asyncio
import aiohttp
import random

async def fetch_data(session, task_id):
    url = f"https://jsonplaceholder.typicode.com/todos/{task_id}"
    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()
            return f"Task {task_id}: {data['title']}"
        else:
            return f"Error fetching task {task_id}"

async def run_in_batches(batch_size, total_tasks):
    async with aiohttp.ClientSession() as session:
        tasks = []
        results = []
        for i in range(1, total_tasks + 1):
            task = fetch_data(session, i)
            tasks.append(task)
            if len(tasks) == batch_size:
                batch_results = await asyncio.gather(*tasks)
                print(f'Batch {i}', batch_results)
                results.extend(batch_results)
                tasks = []
        if tasks:
            batch_results = await asyncio.gather(*tasks)
            results.extend(batch_results)
    return results

async def main():
    batch_size = 3
    total_tasks = 10
    results = await run_in_batches(batch_size, total_tasks)
    for result in results:
        print(result)

# Use this if you're running in a script
await main()

## Tasks out of order

In [ ]:
import asyncio
import aiohttp

async def fetch_data(session, task_id):
    url = f"https://jsonplaceholder.typicode.com/todos/{task_id}"
    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()
            return f"Task {task_id}: {data['title']}"
        else:
            return f"Error fetching task {task_id}"

async def run_in_batches(batch_size, total_tasks):
    async with aiohttp.ClientSession() as session:
        tasks = []
        results = []
        for i in range(1, total_tasks + 1):
            task = fetch_data(session, i)
            tasks.append(task)
            if len(tasks) == batch_size:
                for completed_task in asyncio.as_completed(tasks):
                    result = await completed_task
                    results.append(result)
                tasks = []
        if tasks:  # If there are any leftover tasks after the last full batch
            for completed_task in asyncio.as_completed(tasks):
                result = await completed_task
                results.append(result)
    return results

async def main():
    batch_size = 3
    total_tasks = 10
    results = await run_in_batches(batch_size, total_tasks)
    for result in results:
        print(result)

# Use this if you're running in a script
await main()